In [2]:
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim 
from torchvision import datasets, transforms
from torch.utils.data import Subset
import torch
import os
import pandas as pd
import numpy as np
import cv2
import collections
import time
from torchmetrics import Recall
from torchmetrics import Accuracy
from preprocess import *
import torchvision

Ultralytics YOLOv8.2.2 🚀 Python-3.8.0 torch-2.2.1 CPU (Apple M1)
Setup complete ✅ (8 CPUs, 16.0 GB RAM, 271.0/460.4 GB disk)


In [3]:
new_label={
    0:0,
    1:1,
    2:1,
    3:2,
    4:2
        }

In [4]:
Xtrain,Ytrain,Xtest,Ytest,Xval,Yval  = Preprocess_GD().pipeline()

Processing images class: 0
Processing images class: 1
Processing images class: 2
Processing images class: 3
Processing images class: 4
Processing images class: 0
Processing images class: 1
Processing images class: 2
Processing images class: 3
Processing images class: 4
Processing images class: 0
Processing images class: 1
Processing images class: 2
Processing images class: 3
Processing images class: 4


In [5]:
# Configuration settings
class Config:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    num_epochs = 50
    num_classes = 3
    learning_rate = 1e-4
    model = torchvision.models.resnet18(weights='IMAGENET1K_V1')
    lr_decay = 0.1
    patience = 5
    batch_size = 32
    criterion = nn.CrossEntropyLoss()

config = Config()

In [6]:
model = config.model
model.fc = nn.Linear(model.fc.in_features, 3)

In [7]:
model.load_state_dict(torch.load("/Users/namle/Desktop/SegJSW/bestresnet.pt",map_location=torch.device('cpu')))

<All keys matched successfully>

In [23]:
def minibatch(*tensors, **kwargs):

    batch_size = kwargs['batch_size']

    if len(tensors) == 1:
        tensor = tensors[0]
        for i in range(0, len(tensor), batch_size):
            yield tensor[i:i + batch_size]
    else:
        for i in range(0, len(tensors[0]), batch_size):
            yield tuple(x[i:i + batch_size] for x in tensors)

In [24]:
Ytrain_preds = torch.tensor([])
Yval_preds = torch.tensor([])
Ytest_preds = torch.tensor([])

In [25]:
for (minibatch_num,(Xbtrain)) \
                      in enumerate(minibatch(Xtrain,
                                                      batch_size=32)):

    Ytrain_preds = torch.concat([Ytrain_preds,model(Xbtrain)],dim = 0)

In [ ]:
for (minibatch_num,(Xbval)) \
                      in enumerate(minibatch(Xval,
                                                      batch_size=32)):
    Yval_preds = torch.concat([Yval_preds,model(Xbval)],dim = 0)

In [ ]:
for (minibatch_num,(Xbtest)) \
                      in enumerate(minibatch(Xtest,
                                                      batch_size=32)):
    Ytest_preds = torch.concat([Ytest_preds,model(Xbtest)],dim = 0)

In [16]:
preds = model(Xtrain[:50])

In [18]:
preds.size()

torch.Size([50, 3])

In [19]:
t = model(Xtrain[:10])
t.size()

torch.Size([10, 3])

In [20]:
preds = torch.concat([preds,t],dim=0)

In [21]:
preds.size()

torch.Size([60, 3])

In [14]:
Xtrain.size()

torch.Size([5778, 3, 224, 224])

In [9]:
preds.size()

torch.Size([50, 3])

In [10]:
yolo_process = Preprocess_yolo()

In [11]:
maskch,upper_left,upper_right,lower_left,lower_right = yolo_process.pipeline(Xtrain[:50])

WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.399999999999999' in the future. Please use 'show_labels' instead.
WARNING ⚠️ 'boxes' is deprecated and will be removed in 'ultralytics 8.399999999999999' in the future. Please use 'show_boxes' instead.

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 252.0. Dividing input by 255.
0: 224x224 1 femur, 1 tibia, 267.6ms
1: 224x224 1 femur, 1 tibia, 267.6ms
2: 224x224 1 femur, 1 tibia, 267.6ms
3: 224x224 1 femur, 1 tibia, 267.6ms
4: 224x224 1 femur, 1 tibia, 267.6ms
5: 224x224 1 femur, 1 tibia, 267.6ms
6: 224x224 1 femur, 1 tibia, 267.6ms
7: 224x224 1 femur, 1 tibia, 267.6ms
8: 224x224 1 femur, 1 tibia, 267.6ms
9: 224x224 1 femur, 1 tibia, 267.6ms
10: 224x224 1 femur, 1 tibia, 267.6ms
11: 224x224 1 femur, 1 tibia, 267.6ms
12: 224x224 1 femur, 1 tibia, 267.6ms
13: 224x224 1 femur, 1 tibia, 267.6ms
14: 224x224 1 femur, 1 tibia, 267.6ms
15: 224x224 1 femur, 1 tibia, 267.6ms
16: 224x224 1 f

In [13]:
len(upper_left)

71